# Projeto Abelho Trigona Spinipes

## Processar variáveis

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import glob
import shutil
import rasterio
import numpy as np
from tqdm import tqdm
from rasterio.warp import calculate_default_transform, reproject, Resampling

def extrair_variaveis_bioclimaticas(arquivo_tif, diretorio_saida):
    """
    Extrai as 19 variáveis bioclimáticas de um único arquivo .tif do WorldClim
    e salva cada uma como um arquivo separado.
    
    Args:
        arquivo_tif: Caminho para o arquivo .tif multi-banda do WorldClim
        diretorio_saida: Diretório onde salvar os arquivos individuais
    """
    os.makedirs(diretorio_saida, exist_ok=True)
    
    print(f"Processando arquivo: {arquivo_tif}")
    
    try:
        with rasterio.open(arquivo_tif) as src:
            # Verificar número de bandas
            num_bandas = src.count
            print(f"Número de bandas detectadas: {num_bandas}")
            
            if num_bandas == 1:
                print("AVISO: Este arquivo tem apenas uma banda. Pode não ser um arquivo multi-variável.")
                
                # Extrair nome da variável do nome do arquivo
                nome_arquivo = os.path.basename(arquivo_tif)
                if "bio" in nome_arquivo.lower():
                    # Tentar extrair número da variável bioclimática
                    partes = nome_arquivo.lower().split("bio")
                    if len(partes) > 1:
                        num_var = ''.join(filter(str.isdigit, partes[1]))
                        if num_var:
                            novo_nome = f"bio{num_var}.tif"
                            arquivo_saida = os.path.join(diretorio_saida, novo_nome)
                            
                            # Copiar arquivo
                            shutil.copy(arquivo_tif, arquivo_saida)
                            print(f"Copiado: {nome_arquivo} -> {novo_nome}")
                        else:
                            print(f"Não foi possível extrair número da variável de {nome_arquivo}")
                else:
                    print(f"Arquivo não parece ser uma variável bioclimática: {nome_arquivo}")
            else:
                # Processar arquivo multi-banda
                meta = src.meta.copy()
                
                # Atualizar metadados para arquivos de saída (uma banda)
                meta.update({
                    'count': 1,
                    'driver': 'GTiff',
                    'compress': 'lzw'
                })
                
                # Extrair cada banda como um arquivo separado
                for i in range(1, num_bandas + 1):
                    banda = src.read(i)
                    
                    # Nome do arquivo de saída
                    arquivo_saida = os.path.join(diretorio_saida, f"bio{i}.tif")
                    
                    # Salvar banda como arquivo separado
                    with rasterio.open(arquivo_saida, 'w', **meta) as dst:
                        dst.write(banda, 1)
                    
                    print(f"Extraído: banda {i} -> bio{i}.tif")
                
                print(f"Extração concluída: {num_bandas} variáveis salvas em {diretorio_saida}")
    
    except Exception as e:
        print(f"ERRO ao processar {arquivo_tif}: {str(e)}")

def padronizar_nomes_arquivos(diretorio_origem, diretorio_destino):
    """
    Padroniza os nomes dos arquivos .tif para o formato bio1.tif, bio2.tif, etc.
    
    Args:
        diretorio_origem: Diretório com os arquivos originais
        diretorio_destino: Diretório onde salvar os arquivos renomeados
    """
    os.makedirs(diretorio_destino, exist_ok=True)
    
    print(f"Padronizando nomes dos arquivos em {diretorio_origem}")
    
    # Padrões de nomes conhecidos do WorldClim
    padroes = [
        # WorldClim v2.1
        {"prefixo": "wc2.1_", "separador": "bio"},
        # CMIP6
        {"prefixo": "ssp", "separador": "bio"},
        {"prefixo": "rcp", "separador": "bio"},
        # Outros formatos possíveis
        {"prefixo": "", "separador": "bio"}
    ]
    
    arquivos_processados = 0
    
    for arquivo in glob.glob(os.path.join(diretorio_origem, "*.tif")):
        nome_arquivo = os.path.basename(arquivo)
        nome_base = os.path.splitext(nome_arquivo)[0]
        
        # Tentar extrair número da variável bioclimática
        num_var = None
        
        for padrao in padroes:
            if padrao["separador"] in nome_base.lower():
                partes = nome_base.lower().split(padrao["separador"])
                if len(partes) > 1:
                    # Extrair dígitos do final
                    num_var = ''.join(filter(str.isdigit, partes[1]))
                    if num_var:
                        break
        
        if num_var:
            novo_nome = f"bio{num_var}.tif"
            arquivo_saida = os.path.join(diretorio_destino, novo_nome)
            
            # Copiar arquivo com novo nome
            shutil.copy(arquivo, arquivo_saida)
            print(f"Renomeado: {nome_arquivo} -> {novo_nome}")
            arquivos_processados += 1
        else:
            print(f"Não foi possível extrair número da variável de {nome_arquivo}")
    
    print(f"Padronização concluída: {arquivos_processados} arquivos processados")

def verificar_consistencia(dir_atual, dir_futuro):
    """
    Verifica se os mesmos arquivos existem em ambos os diretórios e têm as mesmas dimensões
    
    Args:
        dir_atual: Diretório com variáveis do clima atual
        dir_futuro: Diretório com variáveis do clima futuro
    
    Returns:
        Lista de variáveis em comum
    """
    print(f"Verificando consistência entre {dir_atual} e {dir_futuro}")
    
    arquivos_atual = set(os.path.basename(f) for f in glob.glob(os.path.join(dir_atual, "*.tif")))
    arquivos_futuro = set(os.path.basename(f) for f in glob.glob(os.path.join(dir_futuro, "*.tif")))
    
    # Verificar arquivos em comum
    comuns = arquivos_atual & arquivos_futuro
    print(f"Variáveis em comum: {len(comuns)}")
    
    if len(comuns) == 0:
        print("ERRO: Nenhuma variável em comum encontrada!")
        return []
    
    # Verificar dimensões
    problemas = []
    for arquivo in comuns:
        try:
            with rasterio.open(os.path.join(dir_atual, arquivo)) as src_atual:
                with rasterio.open(os.path.join(dir_futuro, arquivo)) as src_futuro:
                    if src_atual.shape != src_futuro.shape:
                        print(f"ALERTA: {arquivo} tem dimensões diferentes entre cenários")
                        print(f"  Atual: {src_atual.shape}, Futuro: {src_futuro.shape}")
                        problemas.append(arquivo)
                    if src_atual.crs != src_futuro.crs:
                        print(f"ALERTA: {arquivo} tem CRS diferentes entre cenários")
                        problemas.append(arquivo)
        except Exception as e:
            print(f"ERRO ao verificar {arquivo}: {str(e)}")
            problemas.append(arquivo)
    
    # Remover arquivos problemáticos da lista
    comuns_ok = [arq for arq in comuns if arq not in problemas]
    
    if problemas:
        print(f"ALERTA: {len(problemas)} variáveis com problemas foram removidas da lista")
    
    print(f"Variáveis consistentes: {len(comuns_ok)}")
    return sorted(list(comuns_ok))

def criar_metadados(variaveis, arquivo_saida="metadados_variaveis.md"):
    """
    Cria um arquivo de metadados para as variáveis bioclimáticas
    
    Args:
        variaveis: Lista de nomes de arquivos das variáveis
        arquivo_saida: Nome do arquivo de saída
    """
    descricoes = {
        "bio1.tif": "Temperatura Média Anual",
        "bio2.tif": "Amplitude Média Diurna (média mensal de (temp max - temp min))",
        "bio3.tif": "Isotermalidade (bio2/bio7) (×100)",
        "bio4.tif": "Sazonalidade da Temperatura (desvio padrão ×100)",
        "bio5.tif": "Temperatura Máxima do Mês Mais Quente",
        "bio6.tif": "Temperatura Mínima do Mês Mais Frio",
        "bio7.tif": "Amplitude Térmica Anual (bio5-bio6)",
        "bio8.tif": "Temperatura Média do Trimestre Mais Úmido",
        "bio9.tif": "Temperatura Média do Trimestre Mais Seco",
        "bio10.tif": "Temperatura Média do Trimestre Mais Quente",
        "bio11.tif": "Temperatura Média do Trimestre Mais Frio",
        "bio12.tif": "Precipitação Anual",
        "bio13.tif": "Precipitação do Mês Mais Úmido",
        "bio14.tif": "Precipitação do Mês Mais Seco",
        "bio15.tif": "Sazonalidade da Precipitação (coeficiente de variação)",
        "bio16.tif": "Precipitação do Trimestre Mais Úmido",
        "bio17.tif": "Precipitação do Trimestre Mais Seco",
        "bio18.tif": "Precipitação do Trimestre Mais Quente",
        "bio19.tif": "Precipitação do Trimestre Mais Frio"
    }
    
    with open(arquivo_saida, "w") as f:
        f.write("# Metadados das Variáveis Bioclimáticas\n\n")
        f.write("| Arquivo | Descrição | Unidade |\n")
        f.write("|---------|-----------|--------|\n")
        
        for var in sorted(variaveis):
            nome_base = os.path.basename(var)
            descricao = descricoes.get(nome_base, "Descrição não disponível")
            unidade = "°C" if int(nome_base.replace("bio", "").split(".")[0]) < 12 else "mm"
            f.write(f"| {nome_base} | {descricao} | {unidade} |\n")
    
    print(f"Metadados criados: {arquivo_saida}")

def main():
    """Função principal para processar arquivos bioclimáticos"""
    # Diretórios
    dir_downloads = "downloads"
    dir_clima_atual = "clima_atual"
    dir_clima_futuro = "clima_futuro"
    
    # Criar diretórios se não existirem
    os.makedirs(dir_downloads, exist_ok=True)
    os.makedirs(dir_clima_atual, exist_ok=True)
    os.makedirs(dir_clima_futuro, exist_ok=True)
    
    # Perguntar ao usuário o que deseja fazer
    print("\n===== PROCESSADOR DE VARIÁVEIS BIOCLIMÁTICAS =====\n")
    print("Escolha uma opção:")
    print("1. Extrair variáveis de arquivo .tif multi-banda")
    print("2. Padronizar nomes de arquivos existentes")
    print("3. Verificar consistência entre clima atual e futuro")
    print("4. Criar metadados para as variáveis")
    print("5. Executar todo o processo")
    print("6. Processar pastas que já contêm arquivos bio*.tif")
    print("0. Sair")
    
    opcao = input("\nOpção: ")
    
    if opcao == "1":
        arquivo_tif = input("Caminho para o arquivo .tif multi-banda: ")
        diretorio_saida = input("Diretório de saída [clima_atual]: ") or "clima_atual"
        extrair_variaveis_bioclimaticas(arquivo_tif, diretorio_saida)
    
    elif opcao == "2":
        diretorio_origem = input("Diretório com arquivos originais: ")
        diretorio_destino = input("Diretório de destino para arquivos padronizados: ")
        padronizar_nomes_arquivos(diretorio_origem, diretorio_destino)
    
    elif opcao == "3":
        dir_atual = input("Diretório com clima atual [clima_atual]: ") or "clima_atual"
        dir_futuro = input("Diretório com clima futuro [clima_futuro]: ") or "clima_futuro"
        variaveis_comuns = verificar_consistencia(dir_atual, dir_futuro)
        
        # Salvar lista de variáveis em comum
        if variaveis_comuns:
            with open("variaveis_usadas.txt", "w") as f:
                for var in variaveis_comuns:
                    f.write(f"{var}\n")
            print(f"Lista de variáveis salva em variaveis_usadas.txt")
    
    elif opcao == "4":
        dir_variaveis = input("Diretório com variáveis [clima_atual]: ") or "clima_atual"
        variaveis = [os.path.basename(f) for f in glob.glob(os.path.join(dir_variaveis, "*.tif"))]
        arquivo_saida = input("Nome do arquivo de saída [metadados_variaveis.md]: ") or "metadados_variaveis.md"
        criar_metadados(variaveis, arquivo_saida)
    
    elif opcao == "5":
        # Perguntar caminhos dos arquivos
        arquivo_atual = input("Caminho para o arquivo .tif do clima atual: ")
        arquivo_futuro = input("Caminho para o arquivo .tif do clima futuro: ")
        
        # Extrair variáveis
        if os.path.exists(arquivo_atual):
            if os.path.isdir(arquivo_atual):
                print(f"AVISO: {arquivo_atual} é um diretório, não um arquivo .tif")
            else:
                extrair_variaveis_bioclimaticas(arquivo_atual, dir_clima_atual)
        else:
            print(f"ERRO: Arquivo {arquivo_atual} não encontrado")
        
        if os.path.exists(arquivo_futuro):
            if os.path.isdir(arquivo_futuro):
                print(f"AVISO: {arquivo_futuro} é um diretório, não um arquivo .tif")
            else:
                extrair_variaveis_bioclimaticas(arquivo_futuro, dir_clima_futuro)
        else:
            print(f"ERRO: Arquivo {arquivo_futuro} não encontrado")
        
        # Verificar consistência
        variaveis_comuns = verificar_consistencia(dir_clima_atual, dir_clima_futuro)
        
        # Salvar lista de variáveis em comum
        if variaveis_comuns:
            with open("variaveis_usadas.txt", "w") as f:
                for var in variaveis_comuns:
                    f.write(f"{var}\n")
            print(f"Lista de variáveis salva em variaveis_usadas.txt")
            
            # Criar metadados
            criar_metadados(variaveis_comuns)
    
    elif opcao == "6":
        print("\nProcessando pastas existentes com arquivos bio*.tif")
        dir_atual = input("Diretório com clima atual [clima_atual]: ") or "clima_atual"
        dir_futuro = input("Diretório com clima futuro [clima_futuro]: ") or "clima_futuro"
        
        # Verificar se as pastas existem
        if not os.path.isdir(dir_atual):
            print(f"ERRO: Diretório {dir_atual} não encontrado")
            return
        if not os.path.isdir(dir_futuro):
            print(f"ERRO: Diretório {dir_futuro} não encontrado")
            return
            
        # Verificar consistência
        variaveis_comuns = verificar_consistencia(dir_atual, dir_futuro)
        
        # Salvar lista de variáveis em comum
        if variaveis_comuns:
            with open("variaveis_usadas.txt", "w") as f:
                for var in variaveis_comuns:
                    f.write(f"{var}\n")
            print(f"Lista de variáveis salva em variaveis_usadas.txt")
            
            # Criar metadados
            criar_metadados(variaveis_comuns)
    
    elif opcao == "0":
        print("Saindo...")
    
    else:
        print("Opção inválida!")

if __name__ == "__main__":
    main()


## Ajustar Modelo


In [ ]:
import pandas as pd
import numpy as np
import rasterio
import os
import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
from tqdm import tqdm
from geopy.distance import geodesic

def obter_variaveis_disponiveis():
    """Identifica todas as variáveis bioclimáticas disponíveis em ambos cenários"""
    cli_atual = set(os.path.basename(f) for f in glob.glob("clima_atual/*.tif"))
    cli_futuro = set(os.path.basename(f) for f in glob.glob("clima_futuro/clima_futuro2/*.tif"))
    comuns = sorted(list(cli_atual & cli_futuro))
    print(f"✅ Variáveis em comum: {len(comuns)}")
    for var in comuns:
        print(f"  - {var}")

    # NOVO: Verifica se os arquivos futuros estão corretos e tem o mesmo formato
    try:
        with rasterio.open(os.path.join("clima_atual", comuns[0])) as src_atual:
            shape_atual = src_atual.shape
        with rasterio.open(os.path.join("clima_futuro", comuns[0])) as src_futuro:
            shape_futuro = src_futuro.shape
        
        if shape_atual != shape_futuro:
            print(f"⚠️ ALERTA: Arquivos atuais ({shape_atual}) e futuros ({shape_futuro}) têm dimensões diferentes!")
    except Exception as e:
        print(f"⚠️ Erro ao verificar dimensões dos arquivos: {str(e)}")
    
    return comuns

def stack_rasters(raster_dir, selected_files):
    """Empilha múltiplos rasters em um único array 3D"""
    files = [os.path.join(raster_dir, f) for f in selected_files]
    arrays = []
    meta = None
    for file in tqdm(files, desc=f"📚 Empilhando rasters em {raster_dir}"):
        with rasterio.open(file) as src:
            if meta is None:
                meta = src.meta.copy()
            arrays.append(src.read(1))
    stacked = np.stack(arrays, axis=-1)
    return stacked, meta

def extract_values(coords, raster_stack, meta):
    """Extrai valores de raster para cada coordenada"""
    results = []
    for lat, lon in tqdm(coords, desc="📌 Extraindo valores"):
        row, col = rasterio.transform.rowcol(meta['transform'], lon, lat)
        if 0 <= row < raster_stack.shape[0] and 0 <= col < raster_stack.shape[1]:
            results.append(raster_stack[row, col])
        else:
            results.append([np.nan]*raster_stack.shape[2])
    return np.array(results)

def generate_pseudo_absences(clima, meta, pres_coords, n_samples, min_dist_km=30):
    """Gera pontos de pseudo-ausência longe das presenças conhecidas"""
    absences = []
    attempts = 0
    max_attempts = n_samples * 30  # Aumentado para mais tentativas
    
    print(f"🎯 Gerando {n_samples} pseudo-ausências")
    with tqdm(total=n_samples, desc="🚫 Gerando pseudo-ausências") as pbar:
        while len(absences) < n_samples and attempts < max_attempts:
            # Estratégia para melhor distribuição espacial: dividir em quadrantes
            row = np.random.randint(0, clima.shape[0])
            col = np.random.randint(0, clima.shape[1])
            vals = clima[row, col]
            if np.isnan(vals).any():
                attempts += 1
                continue
            lon, lat = rasterio.transform.xy(meta['transform'], row, col)
            
            # Verifica distância mínima das presenças (agora 30km em vez de 20km)
            if all(geodesic((lat, lon), pc).km > min_dist_km for pc in pres_coords):
                absences.append(vals)
                pbar.update(1)
            attempts += 1
    
    return np.array(absences)

def ajustar_modelo():
    print("🔄 Iniciando ajuste do modelo para reduzir overfitting...")
    
    # Obter variáveis disponíveis
    variaveis = obter_variaveis_disponiveis()
    
    if len(variaveis) <= 2:
        print("⚠️ Apenas 2 variáveis bioclimáticas disponíveis. Para melhor desempenho, adicione mais variáveis.")
    
    # Ler ocorrências
    print("📊 Lendo dados de ocorrência...")
    df = pd.read_csv("ocorrencias.csv")
    coords = list(zip(df["latitude"], df["longitude"]))
    print(f"✅ Presenças: {len(coords)}")
    
    # Empilhar rasters
    print("📚 Empilhando rasters...")
    clima_atual, meta_atual = stack_rasters("clima_atual", variaveis)
    print(f"✅ Clima atual empilhado: {clima_atual.shape}")
    
    # Extrair valores para presenças
    presenças = extract_values(coords, clima_atual, meta_atual)
    
    # Remover presenças com NaN
    nan_mask = np.isnan(presenças).any(axis=1)
    if nan_mask.any():
        print(f"⚠️ Removendo {nan_mask.sum()} presenças com valores NaN")
        presenças = presenças[~nan_mask]
        presenca_coords = [coord for i, coord in enumerate(coords) if not nan_mask[i]]
    else:
        presenca_coords = coords
    
    # Gerar pseudo-ausências
    n_ausencias = len(presenças)
    ausencias = generate_pseudo_absences(clima_atual, meta_atual, presenca_coords, n_ausencias, min_dist_km=30)
    
    # Preparar dados para treinamento
    X = np.vstack((presenças, ausencias))
    y = np.array([1]*len(presenças) + [0]*len(ausencias))
    
    # Normalizar dados
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Dividir em treino/teste
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, test_size=0.2)
    
    # Treinar modelo com parâmetros para prevenir overfitting
    print("🔧 Treinando modelo com parâmetros para reduzir overfitting...")
    
    # Parâmetros mais conservadores para evitar overfitting
    # - Aumentando min_samples_leaf ainda mais
    # - Limitando max_depth para evitar árvores muito profundas
    # - Reduzindo número de árvores para evitar memorização
    rf = RandomForestClassifier(
        n_estimators=50,          # Menos árvores (era 100)
        min_samples_leaf=30,      # Muito maior (era 10)
        max_depth=10,             # Mais limitado (era 15)
        max_features='sqrt',
        class_weight='balanced',
        random_state=42
    )
    
    rf.fit(X_train, y_train)
    
    # Avaliar modelo
    test_acc = rf.score(X_test, y_test)
    print(f"✅ Acurácia no teste: {test_acc:.3f}")
    
    # Salvar modelo
    print("💾 Salvando modelo e scaler...")
    joblib.dump(rf, "modelo_ajustado.pkl")
    joblib.dump(scaler, "scaler_ajustado.pkl")
    joblib.dump(variaveis, "variaveis_usadas.pkl")
    
    # Salvar lista de variáveis
    with open("variaveis_usadas.txt", "w") as f:
        for var in variaveis:
            f.write(f"{var}\n")
    
    print("✅ Modelo ajustado! Para gerar os novos mapas, execute:")
    print("1. python gerar_mapas_ajustados.py")
    print("2. python mascarar_brasil.py")
    print("3. python visualizar_mapas.py")

if __name__ == "__main__":
    ajustar_modelo() 

## Gerar Mapas Ajustados


In [ ]:
import numpy as np
import rasterio
import joblib
import os
import glob
from tqdm import tqdm

def carregar_arquivos():
    """Carrega modelo, scaler e lista de variáveis"""
    if not os.path.exists("modelo_ajustado.pkl"):
        print("❌ Modelo ajustado não encontrado. Execute ajustar_modelo.py primeiro.")
        return None, None, None
    
    modelo = joblib.load("modelo_ajustado.pkl")
    scaler = joblib.load("scaler_ajustado.pkl")
    
    if os.path.exists("variaveis_usadas.pkl"):
        variaveis = joblib.load("variaveis_usadas.pkl")
    else:
        # Fallback para arquivo de texto
        with open("variaveis_usadas.txt", "r") as f:
            variaveis = [line.strip() for line in f.readlines()]
    
    return modelo, scaler, variaveis

def stack_rasters(raster_dir, selected_files):
    """Empilha múltiplos rasters em um único array 3D"""
    files = [os.path.join(raster_dir, f) for f in selected_files]
    arrays = []
    meta = None
    shape_ref = None
    
    # Primeiro, verificar as dimensões de todos os arquivos
    print(f"🔍 Verificando consistência dos arquivos em {raster_dir}...")
    for file in files:
        with rasterio.open(file) as src:
            if shape_ref is None:
                shape_ref = src.shape
                print(f"  📏 Dimensões esperadas: {shape_ref}")
            elif src.shape != shape_ref:
                print(f"  ⚠️ ALERTA: {os.path.basename(file)} tem dimensões diferentes: {src.shape}")
                return None, None
    
    # Se passou na verificação, empilhar os rasters
    for file in tqdm(files, desc=f"📚 Empilhando rasters em {raster_dir}"):
        with rasterio.open(file) as src:
            if meta is None:
                meta = src.meta.copy()
            data = src.read(1)
            # Verificar se há valores válidos
            if np.all(np.isnan(data)):
                print(f"  ⚠️ ALERTA: {os.path.basename(file)} contém apenas valores NaN")
            arrays.append(data)
    
    stacked = np.stack(arrays, axis=-1)
    return stacked, meta

def predizer_mapa(modelo, scaler, clima, meta, nome_saida):
    """Prediz probabilidade de ocorrência para cada pixel"""
    print(f"🗺️ Projetando distribuição para: {nome_saida}")
    
    # Preparar dados
    shape_original = clima.shape[:2]
    clima_reshaped = clima.reshape(-1, clima.shape[2])
    
    # Identificar valores válidos (não-NaN)
    valid_mask = ~np.isnan(clima_reshaped).any(axis=1)
    valid_indices = np.where(valid_mask)[0]
    
    # Inicializar mapa de predição com NaN
    pred_map = np.full(clima_reshaped.shape[0], np.nan)
    
    # Aplicar scaler e prever apenas para pixels válidos
    if len(valid_indices) > 0:
        print(f"✅ Predizendo {len(valid_indices)} pixels válidos...")
        valid_data = clima_reshaped[valid_indices]
        valid_data_scaled = scaler.transform(valid_data)
        
        # Processar em lotes para economizar memória
        batch_size = 100000
        n_batches = (len(valid_indices) + batch_size - 1) // batch_size
        
        for i in tqdm(range(n_batches), desc="🔍 Processando em lotes"):
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, len(valid_indices))
            batch_data = valid_data_scaled[start_idx:end_idx]
            batch_preds = modelo.predict_proba(batch_data)[:, 1]
            pred_map[valid_indices[start_idx:end_idx]] = batch_preds
    
    # Redimensionar para formato original
    pred_map = pred_map.reshape(shape_original)
    
    # Salvar resultado
    print(f"💾 Salvando mapa em: {nome_saida}")
    with rasterio.open(nome_saida, "w", **meta) as dst:
        dst.write(pred_map, 1)
    
    return pred_map

def gerar_mapas():
    # Carregar modelo e parâmetros
    print("🔍 Carregando modelo ajustado...")
    modelo, scaler, variaveis = carregar_arquivos()
    
    if modelo is None:
        return
    
    print(f"✅ Modelo carregado com sucesso. Usando {len(variaveis)} variáveis bioclimáticas:")
    for var in variaveis:
        print(f"  - {var}")
    
    # Empilhar clima atual
    print("\n📚 Empilhando clima atual...")
    clima_atual, meta_atual = stack_rasters("clima_atual", variaveis)
    if clima_atual is None:
        print("❌ Erro ao empilhar clima atual. Verifique as dimensões dos arquivos.")
        return
    print(f"✅ Clima atual empilhado: {clima_atual.shape}")
    
    # Escolher qual conjunto de dados futuros usar
    print("\n🔄 Escolha o conjunto de dados futuros:")
    print("1. clima_futuro2 (variáveis do bio1.tif)")
    print("2. clima_futuro3 (variáveis do bio2.tif)")
    opcao = input("Opção (1 ou 2): ").strip()
    
    if opcao == "1":
        dir_futuro = "clima_futuro/clima_futuro2"
        sufixo = "futuro2"
    elif opcao == "2":
        dir_futuro = "clima_futuro/clima_futuro3"
        sufixo = "futuro3"
    else:
        print("❌ Opção inválida!")
        return
    
    # Empilhar clima futuro
    print(f"\n📚 Empilhando clima futuro de {dir_futuro}...")
    clima_futuro, meta_futuro = stack_rasters(dir_futuro, variaveis)
    if clima_futuro is None:
        print("❌ Erro ao empilhar clima futuro. Verifique as dimensões dos arquivos.")
        return
    print(f"✅ Clima futuro empilhado: {clima_futuro.shape}")
    
    # Verificar se as dimensões são compatíveis
    if clima_atual.shape != clima_futuro.shape:
        print(f"❌ Erro: Dimensões incompatíveis entre clima atual ({clima_atual.shape}) e futuro ({clima_futuro.shape})")
        return
    
    # Predizer mapas
    mapa_atual = predizer_mapa(modelo, scaler, clima_atual, meta_atual, f"mapa_predito_atual_{sufixo}.tif")
    mapa_futuro = predizer_mapa(modelo, scaler, clima_futuro, meta_futuro, f"mapa_predito_{sufixo}.tif")
    
    # Calcular mudança
    if mapa_atual is not None and mapa_futuro is not None:
        print("📊 Calculando mapa de mudança (futuro - atual)...")
        mapa_mudanca = mapa_futuro - mapa_atual
        with rasterio.open(f"mapa_mudanca_{sufixo}.tif", "w", **meta_atual) as dst:
            dst.write(mapa_mudanca, 1)
    
    print("\n✅ Mapas gerados com sucesso! Para continuar o fluxo:")
    print(f"1. Execute: python mascarar_brasil.py (usando os mapas com sufixo _{sufixo})")
    print("2. Execute: python visualizar_mapas.py")

if __name__ == "__main__":
    gerar_mapas() 

## Mascarar Brasil

In [ ]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import os
import glob
from tqdm import tqdm

def aplicar_mascara(mapa_path, shapefile_brasil, saida_path):
    print(f"🗺️ Aplicando máscara ao arquivo: {mapa_path}")

    # Verifica se os arquivos existem
    if not os.path.exists(mapa_path):
        print(f"❌ Arquivo de mapa não encontrado: {mapa_path}")
        return False
    if not os.path.exists(shapefile_brasil):
        print(f"❌ Shapefile não encontrado: {shapefile_brasil}")
        return False

    # Lê o shapefile do Brasil
    try:
        print(f"📂 Lendo shapefile: {shapefile_brasil}")
        brasil = gpd.read_file(shapefile_brasil)
        geoms = brasil.geometry.values
        print(f"✅ Shapefile carregado com {len(brasil)} feições")
    except Exception as e:
        print(f"❌ Erro ao ler o shapefile: {str(e)}")
        return False

    # Abre o raster de entrada
    try:
        print(f"📊 Lendo raster: {mapa_path}")
        with rasterio.open(mapa_path) as src:
            print(f"📏 Dimensões do raster: {src.width}x{src.height}")
            print("🔍 Aplicando máscara...")
            out_image, out_transform = mask(src, geoms, crop=True)
            out_meta = src.meta.copy()
    except Exception as e:
        print(f"❌ Erro ao processar o raster: {str(e)}")
        return False

    # Atualiza os metadados
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform,
        "nodata": src.nodata or -9999
    })

    # Salva o novo arquivo mascarado
    try:
        with rasterio.open(saida_path, "w", **out_meta) as dest:
            dest.write(out_image)
        print(f"✅ Mapa salvo com máscara: {saida_path}")
        return True
    except Exception as e:
        print(f"❌ Erro ao salvar o resultado: {str(e)}")
        return False

def detectar_mapas_gerados():
    """Detecta automaticamente os mapas gerados pelo gerar_mapas_ajustados.py"""
    padroes = [
        "mapa_predito_atual_*.tif",
        "mapa_predito_futuro*.tif", 
        "mapa_mudanca_*.tif"
    ]
    
    arquivos_encontrados = []
    for padrao in padroes:
        arquivos = glob.glob(padrao)
        arquivos_encontrados.extend(arquivos)
    
    # Remove duplicatas e ordena
    return sorted(list(set(arquivos_encontrados)))

if __name__ == "__main__":
    # Caminho para o shapefile
    shapefile_brasil = os.path.join("BR_UF_2024", "BR_UF_2024.shp")
    
    # Detectar mapas gerados automaticamente
    arquivos = detectar_mapas_gerados()
    
    if not arquivos:
        print("❌ Nenhum mapa gerado encontrado!")
        print("Execute primeiro: python gerar_mapas_ajustados.py")
        exit(1)
    
    print(f"📋 Mapas encontrados para processar:")
    for arquivo in arquivos:
        print(f"  - {arquivo}")
    
    # Processa cada arquivo
    sucessos = 0
    for arquivo in tqdm(arquivos, desc="🗺️ Processando mapas"):
        # Criar nome de saída
        nome_base = os.path.splitext(arquivo)[0]
        saida = f"{nome_base}_brasil.tif"
        
        if aplicar_mascara(arquivo, shapefile_brasil, saida):
            sucessos += 1
        print()  # Linha em branco para separar
    
    # Resumo
    print(f"✅ Concluído: {sucessos}/{len(arquivos)} mapas processados com sucesso")
    
    if sucessos > 0:
        print("\n📂 Arquivos gerados com máscara do Brasil:")
        arquivos_brasil = glob.glob("*_brasil.tif")
        for arquivo in sorted(arquivos_brasil):
            print(f"  ✅ {arquivo}")
        
        print("\n🎯 Próximos passos:")
        print("1. Execute: python visualizar_mapas.py")


## Visualizar Mapas

In [ ]:
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import os
import glob

def detectar_mapas_brasil():
    """Detecta automaticamente os mapas mascarados do Brasil"""
    mapas_encontrados = {}
    
    # Procurar mapas de distribuição atual
    atual_files = glob.glob("mapa_predito_atual_*_brasil.tif")
    if atual_files:
        mapas_encontrados["atual"] = atual_files[0]
    
    # Procurar mapas de distribuição futura
    futuro_files = glob.glob("mapa_predito_futuro*_brasil.tif")
    if futuro_files:
        mapas_encontrados["futuro"] = futuro_files[0]
    
    # Procurar mapas de mudança
    mudanca_files = glob.glob("mapa_mudanca_*_brasil.tif")
    if mudanca_files:
        mapas_encontrados["mudanca"] = mudanca_files[0]
    
    return mapas_encontrados

def visualizar_mapa(tif_path, titulo, salvar=True, escala_padronizada=True):
    with rasterio.open(tif_path) as src:
        data = src.read(1).astype(float)

        # Trata valores nodata
        if src.nodata is not None:
            data[data == src.nodata] = np.nan

        # Obtém valores mínimo e máximo reais para estatísticas
        valid_data = data[~np.isnan(data)]
        if len(valid_data) == 0:
            print(f"❌ Nenhum dado válido encontrado em {titulo}")
            return
            
        v_min_real = np.nanmin(valid_data)
        v_max_real = np.nanmax(valid_data)
        
        # Calcula estatísticas básicas
        mean = np.nanmean(valid_data)
        std = np.nanstd(valid_data)
        
        print(f"📊 Estatísticas para {titulo}:")
        print(f"   - Min: {v_min_real:.4f}, Max: {v_max_real:.4f}, Média: {mean:.4f}, Desvio: {std:.4f}")
        
        # Define escala de visualização
        if escala_padronizada:
            if "mudança" in titulo.lower() or "mudanca" in titulo.lower():
                # Para mapas de mudança: escala simétrica baseada no máximo absoluto global
                abs_max = max(abs(v_min_real), abs(v_max_real))
                # Usar uma escala padrão de -0.8 a +0.8 para mudanças
                v_min, v_max = -0.8, 0.8
                cmap = plt.cm.RdBu_r  # Vermelho = perda, Azul = ganho
                label = 'Mudança na Probabilidade'
                print(f"   📏 Escala padronizada para mudança: {v_min} a {v_max}")
            else:
                # Para mapas de probabilidade: escala padronizada 0-1
                v_min, v_max = 0.0, 1.0
                cmap = plt.cm.viridis
                label = 'Probabilidade de Adequabilidade'
                print(f"   📏 Escala padronizada para probabilidade: {v_min} a {v_max}")
        else:
            # Escala dinâmica baseada nos dados
            v_min, v_max = v_min_real, v_max_real
            if "mudança" in titulo.lower() or "mudanca" in titulo.lower():
                cmap = plt.cm.RdBu_r
                label = 'Mudança na Probabilidade'
            else:
                cmap = plt.cm.viridis
                label = 'Probabilidade de Adequabilidade'
            print(f"   📏 Escala dinâmica: {v_min:.4f} a {v_max:.4f}")
        
        # Configura visualização
        plt.figure(figsize=(12, 10))
        
        # Cria o mapa
        im = plt.imshow(data, cmap=cmap, vmin=v_min, vmax=v_max)
        
        # Adiciona título e legenda
        plt.title(titulo, fontsize=16, pad=20)
        plt.axis('off')
        cbar = plt.colorbar(im, label=label, shrink=0.7)
        cbar.ax.tick_params(labelsize=12)
        
        # Adiciona informações sobre valores reais e escala
        info_text = f"Valores reais: {v_min_real:.4f} a {v_max_real:.4f}\nMédia: {mean:.4f} ± {std:.4f}"
        if escala_padronizada:
            info_text += f"\nEscala padronizada: {v_min} a {v_max}"
        
        plt.annotate(info_text, 
                     xy=(0.02, 0.02), xycoords='figure fraction', 
                     fontsize=11, color='black', backgroundcolor='white',
                     bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
        
        plt.tight_layout()
        
        # Salva o mapa em alta resolução para o trabalho acadêmico
        if salvar:
            nome_arquivo = f"{titulo.replace(' ', '_').replace('(', '').replace(')', '')}.png"
            plt.savefig(nome_arquivo, dpi=300, bbox_inches='tight')
            print(f"💾 Mapa salvo como: {nome_arquivo}")
        
        plt.show()

def visualizar_mapa_diferenca(atual_path, futuro_path, escala_padronizada=True):
    """Cria um mapa de diferença entre distribuição futura e atual"""
    
    if not os.path.exists(atual_path) or not os.path.exists(futuro_path):
        print("❌ Arquivos de mapa não encontrados para calcular diferença")
        return
    
    with rasterio.open(atual_path) as src_atual, rasterio.open(futuro_path) as src_futuro:
        data_atual = src_atual.read(1).astype(float)
        data_futuro = src_futuro.read(1).astype(float)
        
        # Trata valores nodata
        if src_atual.nodata is not None:
            data_atual[data_atual == src_atual.nodata] = np.nan
        if src_futuro.nodata is not None:
            data_futuro[data_futuro == src_futuro.nodata] = np.nan
        
        # Calcula diferença
        data_diff = data_futuro - data_atual
        
        # Configurar visualização
        plt.figure(figsize=(12, 10))
        
        # Paleta divergente para mudanças (vermelho = perda, azul = ganho)
        cmap = plt.cm.RdBu_r
        
        # Determinar limite para visualização
        valid_diff = data_diff[~np.isnan(data_diff)]
        if len(valid_diff) == 0:
            print("❌ Nenhum dado válido para calcular diferença")
            return
        
        v_min_real = np.nanmin(data_diff)
        v_max_real = np.nanmax(data_diff)
        
        if escala_padronizada:
            # Escala padronizada para mudanças
            v_min, v_max = -0.8, 0.8
        else:
            # Escala dinâmica simétrica
            abs_max = max(abs(v_min_real), abs(v_max_real))
            v_min, v_max = -abs_max, abs_max
        
        im = plt.imshow(data_diff, cmap=cmap, vmin=v_min, vmax=v_max)
        
        plt.title("Mudança na Distribuição (Futuro - Atual)", fontsize=16, pad=20)
        plt.axis('off')
        cbar = plt.colorbar(im, label='Mudança na Probabilidade', shrink=0.7)
        cbar.ax.tick_params(labelsize=12)
        
        # Calcular estatísticas da diferença
        mean_diff = np.nanmean(data_diff)
        perc_loss = np.sum(data_diff < -0.1) / np.sum(~np.isnan(data_diff)) * 100
        perc_gain = np.sum(data_diff > 0.1) / np.sum(~np.isnan(data_diff)) * 100
        
        # Adiciona informações sobre valores
        info_text = (f"Valores reais: {v_min_real:.4f} a {v_max_real:.4f}\n"
                     f"Média da mudança: {mean_diff:.4f}\n"
                     f"Área com perda (< -0.1): {perc_loss:.1f}%\n"
                     f"Área com ganho (> 0.1): {perc_gain:.1f}%")
        
        if escala_padronizada:
            info_text += f"\nEscala padronizada: {v_min} a {v_max}"
        
        plt.annotate(info_text, xy=(0.02, 0.02), xycoords='figure fraction', 
                     fontsize=11, color='black', backgroundcolor='white',
                     bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
        
        plt.tight_layout()
        
        # Salva o mapa
        plt.savefig("Mudanca_na_Distribuicao.png", dpi=300, bbox_inches='tight')
        print(f"💾 Mapa de mudança salvo como: Mudanca_na_Distribuicao.png")
        
        plt.show()

if __name__ == "__main__":
    # Detectar mapas automaticamente
    mapas = detectar_mapas_brasil()
    
    if not mapas:
        print("❌ Nenhum mapa mascarado encontrado!")
        print("Execute primeiro:")
        print("1. python gerar_mapas_ajustados.py")
        print("2. python mascarar_brasil.py")
        exit(1)
    
    print("📂 Mapas detectados:")
    for tipo, arquivo in mapas.items():
        print(f"  - {tipo}: {arquivo}")
    
    # Perguntar sobre padronização
    print("\n🎨 Escolha o tipo de escala para visualização:")
    print("1. Escala padronizada (0-1 para probabilidades, -0.8 a +0.8 para mudanças)")
    print("2. Escala dinâmica (baseada nos valores dos dados)")
    opcao_escala = input("Opção (1 ou 2) [padrão=1]: ").strip() or "1"
    
    escala_padronizada = opcao_escala == "1"
    
    if escala_padronizada:
        print("✅ Usando escalas padronizadas para comparação científica")
    else:
        print("✅ Usando escalas dinâmicas para visualização detalhada")
    
    # Visualizar mapas de distribuição
    if "atual" in mapas:
        print(f"\n📍 Exibindo: Distribuição Atual")
        # Extrair sufixo do arquivo para identificar o cenário
        nome_arquivo = os.path.basename(mapas["atual"])
        if "futuro2" in nome_arquivo:
            sufixo = " (cenário futuro2)"
        elif "futuro3" in nome_arquivo:
            sufixo = " (cenário futuro3)"
        else:
            sufixo = ""
        visualizar_mapa(mapas["atual"], f"Distribuição Atual{sufixo}", escala_padronizada=escala_padronizada)
    
    if "futuro" in mapas:
        print(f"\n📍 Exibindo: Distribuição Futura")
        # Extrair sufixo do arquivo para identificar o cenário
        nome_arquivo = os.path.basename(mapas["futuro"])
        if "futuro2" in nome_arquivo:
            sufixo = " (cenário futuro2)"
        elif "futuro3" in nome_arquivo:
            sufixo = " (cenário futuro3)"
        else:
            sufixo = ""
        visualizar_mapa(mapas["futuro"], f"Distribuição Futura{sufixo}", escala_padronizada=escala_padronizada)
    
    # Visualizar mapa de mudança se disponível
    if "mudanca" in mapas:
        print(f"\n📍 Exibindo: Mapa de Mudança")
        nome_arquivo = os.path.basename(mapas["mudanca"])
        if "futuro2" in nome_arquivo:
            sufixo = " (cenário futuro2)"
        elif "futuro3" in nome_arquivo:
            sufixo = " (cenário futuro3)"
        else:
            sufixo = ""
        visualizar_mapa(mapas["mudanca"], f"Mudança na Distribuição{sufixo}", escala_padronizada=escala_padronizada)
    
    # Se temos atual e futuro, calcular diferença manualmente se não existe mapa de mudança
    elif "atual" in mapas and "futuro" in mapas:
        print(f"\n📍 Calculando mapa de diferença entre distribuição futura e atual")
        visualizar_mapa_diferenca(mapas["atual"], mapas["futuro"], escala_padronizada=escala_padronizada)
